<h1>Partie 1 : Importation de données vers une base de données sql</h1>

In [11]:
# IMPORTS NECESSAIRES
from dotenv import load_dotenv
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [12]:
# CHARGE LES VARIABLES D'ENVIRONNEMENT STOCKEES DANS LE FICHIER .env
load_dotenv()
host = os.getenv('HOST')

In [13]:
# CREATE_ENGINE CREER LA CONNEXION A LA BDD
# sert d'interface entre votre application Python et la base de données. Il gère les connexions et l'exécution des requêtes SQL.
engine = create_engine(f'mysql+pymysql://laura:Root12345+@{host}:3306/sales')
conn = engine.connect()

In [14]:
# CREER ET GERE LA SESSION QUI INTERAGIT AVEC LA BDD.
# GENERE DES INSTANCES DE SESSIONS CONFIGURE AVEC engine.
Session = sessionmaker(bind=engine)
session = Session()

In [15]:
# LIRE LES CSV
clients_df = pd.read_csv('./data-csv/data-csv/clients.csv', delimiter=';')
produits_df = pd.read_csv('./data-csv/data-csv/produits_sous-categorie.csv', delimiter=';')
ventes_df = pd.read_csv('./data-csv/data-csv/ventes.csv', delimiter=';')

In [51]:
# SUPPRIMER LES DOUBLONS BASES SUR LA COLONNE 'client_id' AVANT l'INSERTION DANS LA TABLES clients
client_sans_doublons = clients_df.drop_duplicates(subset=['client_id'])

In [46]:
# SUPPRIMER LES DOUBLONS BASES SUR LA COLONNE 'product_id' AVANT l'INSERTION DANS LA TABLES produits
produits_df_unique = produits_df.drop_duplicates(subset=['product_id'])

In [48]:
# SUPPRIMER LES DOUBLONS BASES SUR LA COLONNE 'session_id' AVANT l'INSERTION DANS LA TABLES ventes
ventes_df_unique = ventes_df.drop_duplicates(subset=['session_id'])

In [49]:
# FONCTION UTILISEE POUR CREER UNE CLASSE DE BASE QUI SERT A DEFINIR LES CLASSES REPRESENTANT LES TABLES DANS LA BDD
Base = declarative_base()

# JE DEFINIE MAINTENANT LES CLASSES Clients, Produits, Ventes. CHAQUE CLASSE REPRESENTE UNE TABLE DE LA BDD.
# LES COLONNES DES TABLES SONT DEFINIES COMME DES ATTRIBUTS DE CLASSE, AVEC LES TYPES DE DONNEES SQL APPROPRIES.
class Clients(Base):
    __tablename__ = 'clients'
    client_id = Column(String(10),primary_key=True )
    sex = Column(String(2))
    birth = Column(Integer)

class Produits(Base):
    __tablename__ = 'produits'
    product_id = Column(String(10),primary_key=True)
    category = Column(String(50))
    sub_category = Column(String(50))
    price = Column(Float)
    stock_quantity = Column(Integer)

class Ventes(Base):
    __tablename__ = 'ventes'
    product_id = Column(String(10))
    date = Column(String(50))
    session_id = Column(String(50),primary_key=True)
    client_id = Column(String(50))
    quantity_sold = Column(Integer)

# CREATION DES TABLES DEFINIES PAR LES CLASSES DANS LA BDD SI ELLES N'EXISTENT PAS DEJA;
Base.metadata.create_all(engine)

/tmp/ipykernel_123599/2530631333.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [50]:
try:
    # PREPARER LES DONNEES POUR L'INSERTION DANS clients
    client_data = [{
        'client_id': row[0],
        'sex': row[1],
        'birth': row[2]
    } for index, row in client_sans_doublons.iterrows()]


    # PREPARER LES DONNEES POUR L'INSERTION DANS produits
    produits_data = [{
        'product_id': row[0],
        'category': row[1],
        'sub_category': row[2],
        # Conversion du prix : remplace les virgules par des points et convertir en float
        'price': float(str(row[3]).replace(',', '.')),
        'stock_quantity': row[4]
    } for index, row in produits_df_unique.iterrows()]



    # PREPARER LES DONNEES POUR L'INSERTION DANS ventes
    ventes_data = [{
        'product_id': row[0],
        'date': row[1],
        'session_id': row[2],
        'client_id': row[3],
        'quantity_sold': row[4]
    } for index, row in ventes_df_unique.iterrows()]

    # INSERT LES DONNEES EN MASSE DANS CHAQUE TABLE
    session.bulk_insert_mappings(Clients, client_data)
    session.bulk_insert_mappings(Produits, produits_data)
    session.bulk_insert_mappings(Ventes, ventes_data)

    # COMMIT LA SESSION
    session.commit()

except Exception as e:
    # Si une erreur survient, rollback de la session
    print(f"Erreur lors de l'importation des données : {e}")
    session.rollback()

finally:
    # Fermeture de la session
    session.close()

print("Importation des données terminée.")


Importation des données terminée.
